In [1]:
import torch
device = torch.device("cuda")

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

/home/aw82/miniconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
model_weights_by_layer = dict()
for layer in range(32):
    # self attn layers
    model_weights_by_layer[f"{layer}_self_attn_q_proj"] = model.model.layers[layer].self_attn.q_proj.weight.data.cpu().numpy()
    model_weights_by_layer[f"{layer}_self_attn_k_proj"] = model.model.layers[layer].self_attn.k_proj.weight.data.cpu().numpy()
    model_weights_by_layer[f"{layer}_self_attn_v_proj"] = model.model.layers[layer].self_attn.v_proj.weight.data.cpu().numpy()
    model_weights_by_layer[f"{layer}_self_attn_o_proj"] = model.model.layers[layer].self_attn.o_proj.weight.data.cpu().numpy()
    # mlp layers
    model_weights_by_layer[f"{layer}_mlp_gate_proj"] = model.model.layers[layer].mlp.gate_proj.weight.data.cpu().numpy()
    model_weights_by_layer[f"{layer}_mlp_up_proj"] = model.model.layers[layer].mlp.up_proj.weight.data.cpu().numpy()
    model_weights_by_layer[f"{layer}_mlp_down_proj"] = model.model.layers[layer].mlp.down_proj.weight.data.cpu().numpy()
    
model_weights_by_layer["embed_tokens"] = model.model.embed_tokens.weight.data.cpu().numpy()
model_weights_by_layer["lm_head"] = model.lm_head.weight.data.cpu().numpy()

In [4]:
import pickle
with open(f"{model_name}.pickle", 'wb') as handle:
    pickle.dump(model_weights_by_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)